In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from pathlib import Path
import pandas as pd
from spotifuncs import authenticate,create_df_top_songs, append_audio_features,create_similarity_score,get_recommendations, top_artists_from_API, create_df_saved_songs

In [2]:
path = Path("C:/Users/ms101/OneDrive/DataScience_ML/projects/spotify_app")

In [3]:
with open(path / "client_s.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content]

client_id = content[0]
client_secret = content[1]

In [4]:
with open(path / "usernames.txt") as f:
    usernames = f.readlines()
usernames = [x.strip() for x in usernames]

username1 = usernames[0]
username2 = usernames[1]

In [5]:
scope = "user-library-read user-read-recently-played user-top-read playlist-modify-public playlist-read-private playlist-read-collaborative"

In [6]:
redirect_uri = "https://developer.spotify.com/dashboard/applications/4a4e029d299a4241873db8300038bf0a"


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                       client_secret=client_secret)

# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)


# token = util.prompt_for_user_token(username2, scope, client_id, client_secret, redirect_uri)

# if token:
#     sp = spotipy.Spotify(auth=token)
# else:
#     print("Can't get token for", username2)

In [7]:
sp = authenticate(redirect_uri, client_credentials_manager, username2, scope, client_id, client_secret)

In [8]:
##queries
#user top tracks
top_tracks_short = sp.current_user_top_tracks(limit = 50,offset=0,time_range='short_term')
top_tracks_med = sp.current_user_top_tracks(limit = 50,offset=0,time_range='medium_term')
top_tracks_long = sp.current_user_top_tracks(limit = 50,offset=0,time_range='long_term')
#user top artists
top_artists_long = sp.current_user_top_artists(limit = 50, time_range = "long_term")
top_artists_med = sp.current_user_top_artists(limit = 50, time_range = "medium_term")
top_artists_short = sp.current_user_top_artists(limit = 50, time_range = "short_term")
#user recent tracks
#still in BETA phase no spotipy function (workaround??)

#user saved tracks
user_saved_tracks = sp.current_user_saved_tracks(limit = 50)


In [9]:
artists_short_df = top_artists_from_API(top_artists_short)
artists_med_df = top_artists_from_API(top_artists_med)
artists_long_df = top_artists_from_API(top_artists_long)
artists_df = pd.concat([artists_short_df,artists_med_df,artists_long_df])
artists_df["genres"] = artists_df["genres"].apply(lambda x: ",".join(x))
artists_df.drop_duplicates().reset_index(drop = True)
artists_df

,name,id,genres,popularity,uri
0,Emancipator,6HCnsY0Rxi3cg53xreoAIm,"downtempo,electronica,livetronica,nu jazz,trip...",61,spotify:artist:6HCnsY0Rxi3cg53xreoAIm
1,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,"garage rock,modern rock,permanent wave,rock,sh...",86,spotify:artist:7Ln80lUS6He07XvHI8qqHH
2,Masego,3ycxRkcZ67ALN3GQJ57Vig,"alternative r&b,indie soul",74,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig
3,Element Of Crime,3FweAJRBCbUOGR6jULfaRi,"german indie,german pop,german rock,liedermacher",49,spotify:artist:3FweAJRBCbUOGR6jULfaRi
4,Mac Miller,4LLpKhyESsyAXpc4laK94U,"hip hop,pittsburgh rap,pop rap,rap",85,spotify:artist:4LLpKhyESsyAXpc4laK94U
...,...,...,...,...,...
45,Eminem,7dGJo4pcD2V6oG8kP0tJRR,"detroit hip hop,hip hop,rap",92,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR
46,MEUTE,1z5xbcOeFRQXBVDpvRPh8H,"german dance,hamburg electronic,livetronica",53,spotify:artist:1z5xbcOeFRQXBVDpvRPh8H
47,Max Herre,7IpWQKu80qQvyer3LO6SW3,"german alternative rap,german hip hop,german pop",53,spotify:artist:7IpWQKu80qQvyer3LO6SW3
48,Hozier,2FXC3k01G6Gw61bmprjgqS,"irish singer-songwriter,modern rock,pop",80,spotify:artist:2FXC3k01G6Gw61bmprjgqS


In [10]:
artists_short_df

,name,id,genres,popularity,uri
0,Emancipator,6HCnsY0Rxi3cg53xreoAIm,"[downtempo, electronica, livetronica, nu jazz,...",61,spotify:artist:6HCnsY0Rxi3cg53xreoAIm
1,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,"[garage rock, modern rock, permanent wave, roc...",86,spotify:artist:7Ln80lUS6He07XvHI8qqHH
2,Masego,3ycxRkcZ67ALN3GQJ57Vig,"[alternative r&b, indie soul]",74,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig
3,Didula,6c9cqJC0XghPWB6ywd8Uwr,[belarusian pop],35,spotify:artist:6c9cqJC0XghPWB6ywd8Uwr
4,Mac Miller,4LLpKhyESsyAXpc4laK94U,"[hip hop, pittsburgh rap, pop rap, rap]",85,spotify:artist:4LLpKhyESsyAXpc4laK94U
5,Dmitri Kitayenko,3Nfbf48vyOcRJmK5dXoPHg,[],35,spotify:artist:3Nfbf48vyOcRJmK5dXoPHg
6,Jah Khalib,4TjXA854eWs8P5EBgCbQmv,"[kazakh pop, russian gangster rap, russian hip...",61,spotify:artist:4TjXA854eWs8P5EBgCbQmv
7,Frank Sinatra,1Mxqyy3pSjf8kZZL4QVxS0,"[adult standards, easy listening, lounge]",85,spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0
8,Monolink,2I4hRNCYkPKJQlkoEZKjYx,"[electronica, organic house, tech house]",62,spotify:artist:2I4hRNCYkPKJQlkoEZKjYx
9,Lana Del Rey,00FQb4jTyendYWaN8pK0wa,"[art pop, pop]",85,spotify:artist:00FQb4jTyendYWaN8pK0wa


In [11]:
artists_med_df

,name,id,genres,popularity,uri
0,Herman van Veen,5jWFYMXC1s1pj0u22ps4Sh,"[dutch pop, dutch rock, muziek voor kinderen, ...",51,spotify:artist:5jWFYMXC1s1pj0u22ps4Sh
1,Masego,3ycxRkcZ67ALN3GQJ57Vig,"[alternative r&b, indie soul]",74,spotify:artist:3ycxRkcZ67ALN3GQJ57Vig
2,Apache 207,1qQLhymHXFPtP5U8KNKsm6,[german hip hop],75,spotify:artist:1qQLhymHXFPtP5U8KNKsm6
3,Emancipator,6HCnsY0Rxi3cg53xreoAIm,"[downtempo, electronica, livetronica, nu jazz,...",61,spotify:artist:6HCnsY0Rxi3cg53xreoAIm
4,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,"[garage rock, modern rock, permanent wave, roc...",86,spotify:artist:7Ln80lUS6He07XvHI8qqHH
5,Element Of Crime,3FweAJRBCbUOGR6jULfaRi,"[german indie, german pop, german rock, lieder...",49,spotify:artist:3FweAJRBCbUOGR6jULfaRi
6,NUGAT,74yjU6gHx05PeHAokao6DS,[indie deutschrap],52,spotify:artist:74yjU6gHx05PeHAokao6DS
7,Frank Sinatra,1Mxqyy3pSjf8kZZL4QVxS0,"[adult standards, easy listening, lounge]",85,spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0
8,Modeselektor,2jYMYP2SVifgmzNRQJx3SJ,"[electronica, microhouse, minimal techno]",50,spotify:artist:2jYMYP2SVifgmzNRQJx3SJ
9,Antoine de Saint-Exupéry,6hjUG26LEL5FK2pM7pWwph,[reading],43,spotify:artist:6hjUG26LEL5FK2pM7pWwph


In [12]:
artists_long_df

,name,id,genres,popularity,uri
0,Seeed,5ISjkNS17JpCwiFtW80lpV,"[german hip hop, german pop, german reggae, ge...",66,spotify:artist:5ISjkNS17JpCwiFtW80lpV
1,Herman van Veen,5jWFYMXC1s1pj0u22ps4Sh,"[dutch pop, dutch rock, muziek voor kinderen, ...",51,spotify:artist:5jWFYMXC1s1pj0u22ps4Sh
2,Sting,0Ty63ceoRnnJKVEYP0VQpk,"[permanent wave, soft rock, sophisti-pop]",73,spotify:artist:0Ty63ceoRnnJKVEYP0VQpk
3,Lana Del Rey,00FQb4jTyendYWaN8pK0wa,"[art pop, pop]",85,spotify:artist:00FQb4jTyendYWaN8pK0wa
4,Clueso,0iFq8SzyZaaQ2Fn6CLDxqW,"[german hip hop, german indie, german pop, ger...",67,spotify:artist:0iFq8SzyZaaQ2Fn6CLDxqW
5,Monolink,2I4hRNCYkPKJQlkoEZKjYx,"[electronica, organic house, tech house]",62,spotify:artist:2I4hRNCYkPKJQlkoEZKjYx
6,Katie Melua,5uCXJWo3WoXgqv3T1RlAbh,"[uk pop, vocal jazz]",63,spotify:artist:5uCXJWo3WoXgqv3T1RlAbh
7,Charlie Cunningham,78CiW0UJbHspFaVuVexOK6,"[british singer-songwriter, indie anthem-folk,...",62,spotify:artist:78CiW0UJbHspFaVuVexOK6
8,Bausa,13bDjug9N0pyv3ZUINjkDV,"[german hip hop, german pop]",70,spotify:artist:13bDjug9N0pyv3ZUINjkDV
9,Joy Denalane,5vP3nmsaGrondXXS5BvrSH,"[german pop, neo soul]",54,spotify:artist:5vP3nmsaGrondXXS5BvrSH


Working with the saved tracks (only the last 50)

The dictionary follows a different structure than the one for top_tracks, a new function will be necessary. 

Let's first explore the structure and try to access the info we want.

In [13]:
user_saved_tracks["items"][0]["track"].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [14]:
user_saved_tracks["items"][0]["track"]["name"]

'Take Some Time - Emancipator Remix'

In [15]:
user_saved_tracks["items"][0]["track"]["album"]["name"]

'Take Some Time (Emancipator Remix)'

In [16]:
user_saved_tracks["items"][0]["track"]["artists"][0]["name"]

'Wilderado'

In [17]:
user_saved_tracks["items"][0]["track"]["popularity"]

40

In [18]:
user_saved_tracks["items"][0]["track"]["id"]

'28M2ugvRSIa4MIKmiiwNao'

In [19]:
user_saved_tracks["items"][0]["track"]["duration_ms"]

290428

In [20]:
saved_tracks_df = create_df_saved_songs(user_saved_tracks)

In [21]:
saved_tracks_df

,track_name,album,track_id,artist,duration,popularity
0,Take Some Time - Emancipator Remix,Take Some Time (Emancipator Remix),28M2ugvRSIa4MIKmiiwNao,Wilderado,290428,40
1,Time for Space,Baralku,3dJXvBddoH1AGLpKvmbYDA,Emancipator,433613,54
2,Shuggie - Bonus Track,Her,24V1nw6aGxnIVuWHEGH4e1,Her,259519,45
3,Borboletas,Borboletas,5dYAXtlAkKVE2I4PYMyHie,Maple Syrup,145588,45
4,When The Interlude Sounds Like Falling Rain,What Are You Into EP,5lWVqc9kPplvlDmLtFls02,Nandu,300668,42
5,Phía Sau Đôi Mắt,Phía Sau Đôi Mắt,3DJZM2sfbUleDaCLRXWSco,Anh bạn Thành,174206,42
6,Suite No. 2 for Jazz Orchestra: VII. Waltz II,Schostakowitsch: Jazz-Suiten,416revLjetHx8BziPfxhRn,Dmitri Shostakovich,231640,13
7,Carmen,Born To Die,30yDbkiLvSiYx3pox7odMR,Lana Del Rey,248720,39
8,Zukunft,12,2yI3NNFLbKskIBpGuiANjV,AnnenMayKantereit,191506,62
9,Bye Felicia,Studying Abroad,39RwDjX4s7OuUvlutGjBZI,Masego,158112,55


In [22]:
#combine the top_tracks
top_tracks_short_df = append_audio_features(create_df_top_songs(top_tracks_short),sp)
top_tracks_med_df = append_audio_features(create_df_top_songs(top_tracks_med),sp)
top_tracks_long_df = append_audio_features(create_df_top_songs(top_tracks_long),sp)
top_tracks_df = pd.concat([top_tracks_short_df,top_tracks_med_df,top_tracks_long_df]).drop_duplicates().reset_index(drop = True)
top_tracks_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Márquez: Danzón No.2,Música Clásica Esencial: México,4IM73PUWb5Y07VcFOsUlhE,Arturo Márquez,585693,23,0.330,0.267,2,-19.607,0,0.0511,0.85600,0.782000,0.1270,0.2350,102.855
1,Time for Space,Baralku,3dJXvBddoH1AGLpKvmbYDA,Emancipator,433613,54,0.610,0.313,5,-13.138,0,0.0323,0.39000,0.840000,0.0899,0.0870,80.009
2,505,Favourite Worst Nightmare,0BxE4FqsDD1Ot4YuBXwAPp,Arctic Monkeys,253586,79,0.526,0.866,0,-5.822,1,0.0568,0.00287,0.000078,0.0945,0.2480,140.266
3,Mystery Lady (feat. Don Toliver),Mystery Lady (feat. Don Toliver),2Ee9amLUslOwgoJWZEpSSD,Masego,228453,74,0.747,0.494,3,-7.957,0,0.1410,0.39300,0.000000,0.0982,0.6240,76.006
4,Greenland,Safe In the Steep Cliffs,2SPTGg9SC5MT1FwNX4IYfx,Emancipator,191066,56,0.593,0.398,4,-13.301,0,0.0295,0.02730,0.827000,0.2310,0.0989,98.728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,The Harder the Ground,Ø,5hb7SiqKHDlMPzs9HEholc,Gløde,243979,13,0.533,0.240,5,-14.995,1,0.0370,0.92300,0.000239,0.0877,0.0743,126.007
135,Smek - Rey&Kjavik Remix,Smek (Rey&Kjavik Remix),1PyfWp1rg8omX2DOtg65pR,Ÿuma,366006,54,0.892,0.281,2,-13.880,1,0.0435,0.85600,0.878000,0.1950,0.0433,99.984
136,Baby Girl,Baby Girl,6DNTJAvdyDwkmiPm2Wl8At,Bryce Vine,145371,55,0.860,0.592,2,-8.606,1,0.0861,0.16600,0.000609,0.2760,0.6210,107.016
137,Will We Ever Carry On,Chances,2AMAh0IXu1Wwwdw3Jzojqt,Luke Marzec,254549,18,0.638,0.293,0,-15.712,0,0.0337,0.55500,0.505000,0.1110,0.2260,81.802


In [23]:
top_tracks_short_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Márquez: Danzón No.2,Música Clásica Esencial: México,4IM73PUWb5Y07VcFOsUlhE,Arturo Márquez,585693,23,0.330,0.26700,2,-19.607,0,0.0511,0.85600,0.782000,0.1270,0.2350,102.855
1,Time for Space,Baralku,3dJXvBddoH1AGLpKvmbYDA,Emancipator,433613,54,0.610,0.31300,5,-13.138,0,0.0323,0.39000,0.840000,0.0899,0.0870,80.009
2,505,Favourite Worst Nightmare,0BxE4FqsDD1Ot4YuBXwAPp,Arctic Monkeys,253586,79,0.526,0.86600,0,-5.822,1,0.0568,0.00287,0.000078,0.0945,0.2480,140.266
3,Mystery Lady (feat. Don Toliver),Mystery Lady (feat. Don Toliver),2Ee9amLUslOwgoJWZEpSSD,Masego,228453,74,0.747,0.49400,3,-7.957,0,0.1410,0.39300,0.000000,0.0982,0.6240,76.006
4,Greenland,Safe In the Steep Cliffs,2SPTGg9SC5MT1FwNX4IYfx,Emancipator,191066,56,0.593,0.39800,4,-13.301,0,0.0295,0.02730,0.827000,0.2310,0.0989,98.728
5,Frosty The Snowman - Remastered 2006,Bing Crosby - Christmas Classics,35MqcEIVZ7svbBpRt4N30g,Bing Crosby,133413,62,0.519,0.36800,7,-11.622,1,0.0796,0.61700,0.000000,0.0859,0.8540,160.370
6,Lavish Lullaby,Lady Lady,4dulFGnSzdNW3iNdxxzgy7,Masego,197360,57,0.839,0.65800,2,-5.515,0,0.0450,0.25200,0.020200,0.1490,0.8180,147.005
7,Soon It Will Be Cold Enough to Build Fires,Soon It Will Be Cold Enough,7cHRys0Lhk9642dLaPUMkm,Emancipator,180520,52,0.707,0.46700,7,-15.094,0,0.0488,0.84900,0.685000,0.0957,0.4190,94.417
8,The Long Run,The Long Run,7wbozFLc9AaQmC9dSAcCdQ,ELIH,327500,40,0.694,0.57900,11,-11.105,0,0.0403,0.17300,0.061300,0.1380,0.1940,96.013
9,Джадуа,ВЫХОД В СВЕТ,1AC2EZQ4EOzPiDponJAvMh,Jah Khalib,248000,57,0.891,0.62000,11,-6.821,0,0.0835,0.01840,0.610000,0.0975,0.2390,110.008


In [25]:
top_tracks_med_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Wealth,Who Else,5aOlYhQsp75cgPov4yjWIe,Modeselektor,247218,42,0.765,0.4520,1,-12.346,1,0.0856,0.020400,0.000057,0.0727,0.3310,137.972
1,The Finishing,The Ginning,2SCLswvRP8hAYdIKsrLV6h,Stavroz,482983,61,0.773,0.3770,9,-12.653,1,0.0455,0.655000,0.829000,0.1220,0.0363,119.975
2,"Мне нравится, что вы больны не мной","Микаэл таривердиев, музыка из кинофильма ""ирон...",42NLt1tVVwhh4KncxJfRHk,Alla Pugacheva,93786,41,0.574,0.0879,7,-21.303,0,0.0605,0.952000,0.004060,0.1070,0.5150,104.627
3,Hundertquadratmeterherz,Lieder vom Leben und anderen Zuständen,1Nkdfq6FIhqkgUU3LQXwWk,Oliver Jahn & Band,273986,0,0.373,0.3780,9,-8.861,0,0.0314,0.234000,0.000000,0.1980,0.4580,175.790
4,Bind You to My Spell,The World Within,6483OUXil7qHDhI7GV8OwH,Moderator,199857,42,0.702,0.4230,10,-8.375,0,0.0434,0.193000,0.384000,0.1280,0.7340,74.455
5,Lost Minds Office - Original Mix,Incabus,1VbBTopCa3CLbWSqVO4fHf,Maguera,414096,37,0.581,0.3470,1,-12.952,1,0.0502,0.232000,0.672000,0.1090,0.4560,179.978
6,Huldra - Other Version,Huldra,569gNjph2g07MmjtMm6vKm,Gidge,490000,38,0.772,0.5140,10,-11.961,0,0.0754,0.148000,0.906000,0.0934,0.1030,118.015
7,Bläulich,Treppenhaus,2WRTnY0slmFgWcrmEr8dPj,Apache 207,196213,70,0.790,0.7040,10,-7.935,0,0.4170,0.069000,0.000658,0.1130,0.2120,154.007
8,Drop Dead,Drop Dead,3i4CFXy3ezQZEt8V0GA7Uh,Holly Humberstone,177162,64,0.577,0.3930,0,-7.322,1,0.0380,0.586000,0.000181,0.0927,0.1320,89.858
9,Easy Field,Hometown,2tbITVFp25m5DRf3lbsDLA,Stereoclip,259546,46,0.878,0.5530,1,-9.574,1,0.0395,0.794000,0.909000,0.0900,0.0394,108.040


In [27]:
top_tracks_long_df

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,The Finishing,The Ginning,2SCLswvRP8hAYdIKsrLV6h,Stavroz,482983,61,0.773,0.3770,9,-12.653,1,0.0455,0.65500,0.829000,0.1220,0.0363,119.975
1,Moon Over Bourbon Street,The Dream Of The Blue Turtles,7wMYruLZuMs2TLuPXRwSDr,Sting,241080,0,0.450,0.1690,5,-13.306,0,0.0363,0.76800,0.000739,0.1060,0.2130,125.411
2,Welcome to Wonderland,Welcome to Wonderland,3JfHYZKy5JmE5Fv4gDTCiz,Anson Seabra,199131,74,0.703,0.1790,4,-10.906,1,0.0366,0.94900,0.000000,0.1020,0.3460,97.427
3,Riot On An Empty Street,Declaration Of Dependence,0hTN97yQDyleWFrWDca7XY,Kings of Convenience,245493,38,0.429,0.2890,5,-16.064,1,0.0334,0.89800,0.045600,0.1100,0.2930,154.537
4,Was du Liebe nennst,Was Du Liebe nennst,4hKWUzhQWsOkgT6LnDEASe,Bausa,203630,62,0.874,0.4870,2,-8.154,1,0.0641,0.39300,0.000030,0.0524,0.7460,144.956
5,Huldra - Other Version,Huldra,569gNjph2g07MmjtMm6vKm,Gidge,490000,38,0.772,0.5140,10,-11.961,0,0.0754,0.14800,0.906000,0.0934,0.1030,118.015
6,What A Day,BAM BAM,3Zexl3b8tdk5csqUHgTPnZ,Seeed,153759,38,0.435,0.3900,9,-8.826,0,0.0271,0.46900,0.000000,0.1730,0.2000,132.331
7,"Мне нравится, что вы больны не мной","Микаэл таривердиев, музыка из кинофильма ""ирон...",42NLt1tVVwhh4KncxJfRHk,Alla Pugacheva,93786,41,0.574,0.0879,7,-21.303,0,0.0605,0.95200,0.004060,0.1070,0.5150,104.627
8,Scarborough Fair,A New Journey,1U1NkDoPpCSEX1T6ZMWX04,Celtic Woman,192973,56,0.247,0.2330,10,-13.611,1,0.0366,0.91400,0.000025,0.3790,0.1020,123.097
9,Part Eleven,Parts of Life,6MI7XPygeiQlQ4lN9I2b5X,Paul Kalkbrenner,205346,46,0.603,0.5350,9,-11.260,0,0.0339,0.26800,0.792000,0.1750,0.2250,132.040


In [26]:
saved_tracks_df = append_audio_features(saved_tracks_df,sp)

In [22]:
saved_tracks_df.head(10)

,track_name,album,track_id,artist,duration,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,I Fell In Love,Autumn Bells,0zrHx4EhLePXUa8KhGpS3E,Gidge,385726,44,0.539,0.610,8,-10.999,0,0.1340,0.12100,0.809000,0.0857,0.0395,123.218
1,The Beginning,The Beginning,4njhWDhTAjhReWtYkiMH9t,NR:TN,201904,44,0.783,0.714,7,-7.408,1,0.2460,0.24300,0.860000,0.4460,0.0341,125.955
2,Between Breaths,Stateless,4DJGTc1OsgqFsUGI6W8Mtx,Riyoon,479885,43,0.738,0.665,7,-10.441,0,0.0438,0.00748,0.877000,0.5820,0.0347,99.992
3,Berlin Nights,Berlin Nights,6gG1R1bFdJeNc2ERAwXxCb,Vnce Dolanbay,292115,39,0.901,0.457,10,-13.238,0,0.1630,0.22900,0.424000,0.0977,0.5310,127.999
4,Fall Slowly (feat. Ashanti) - Extended Version,Evolution,7wK4pOTZKVaAJ00rziu901,Joyner Lucas,292056,59,0.542,0.453,6,-11.208,0,0.2400,0.20400,0.000007,0.1370,0.0375,88.310
5,Evolution,Evolution,2VopDw2GlF3uwD1kihHmTT,Joyner Lucas,153250,62,0.687,0.819,9,-6.670,0,0.4310,0.21800,0.000000,0.3920,0.5680,81.185
6,Zim Zimma,Evolution,17nPeSliosCi427f0lUb75,Joyner Lucas,239702,70,0.883,0.621,11,-6.063,0,0.2120,0.08710,0.000000,0.4990,0.6760,149.052
7,Money In The Grave (Drake ft. Rick Ross),The Best In The World Pack,5ry2OE6R2zPQFDO85XkgRb,Drake,205426,81,0.831,0.502,10,-4.045,0,0.0460,0.10100,0.000000,0.1220,0.1010,100.541
8,Beifahrersitz,Beifahrersitz,01qOl2pM8emx1sxdBQc05g,LEA,199586,71,0.712,0.774,6,-3.967,0,0.1330,0.34600,0.000000,0.1760,0.4710,159.977
9,Twingo,POP,6vFEvkXOnOTUMgPHxogIRK,Fynn Kliemann,186984,56,0.619,0.638,2,-5.754,0,0.0906,0.29700,0.000001,0.4400,0.3600,141.967


In [23]:
similarity_top_tracks = create_similarity_score(top_tracks_df,top_tracks_df)
similarity_saved_tracks = create_similarity_score(saved_tracks_df,saved_tracks_df)

In [24]:
for track in top_tracks_df["track_name"].sample(5):
    print(track, "\n")
    print(get_recommendations(top_tracks_df,track,similarity_top_tracks))
    print("\n")

MOSKAU 

112        Light Rain
27            Crainte
21     Slap the Ghost
89          Magnifico
22               Zver
Name: track_name, dtype: object


FEUER 

22                          Zver
128       I Mean It (feat. Remo)
4                    Rolls Royce
51                          YAYA
16     Creep - R3HAB Chill Remix
Name: track_name, dtype: object


Nero Forte 

75                                              Holy War
60                                              Unspoken
116                                The Sound of Violence
95     Still A Full Gentle Racket - Angerfist 2019 Refix
103                                           Gold Cobra
Name: track_name, dtype: object


m.A.A.d city 

112       Light Rain
88       Holy Roller
97      m.A.A.d city
107    6 Foot 7 Foot
75          Holy War
Name: track_name, dtype: object


Huldra 

47                                                Huldra
49                                Huldra - Other Version
24     In The (Last) Moment - R

In [25]:
for track in saved_tracks_df["track_name"].sample(5):
    print(track, "\n")
    print(get_recommendations(saved_tracks_df,track,similarity_saved_tracks))
    print("\n")

Idol (feat. Tech N9ne) 

24        Slap the Ghost
41                Omertà
34              Over Now
27    Lost In The Static
38               BURNING
Name: track_name, dtype: object


Light Rain 

26         Full Metal
2     Between Breaths
38            BURNING
29             So Far
0      I Fell In Love
Name: track_name, dtype: object


I Fell In Love 

0                                     I Fell In Love
33                                        Light Rain
4     Fall Slowly (feat. Ashanti) - Extended Version
2                                    Between Breaths
14                    Risky Business - Mathame Remix
Name: track_name, dtype: object


When I'm Small 

32                 YAYA
25    You.Won't.Do.Shit
16             Blackout
10                 ADHD
39                Alarm
Name: track_name, dtype: object


Generation YouPorn - Agia Napa Edit 

29                                            So Far
21                                      Five Minutes
4     Fall Slowly (feat. Ash

In [26]:
##Playlists
#spotify features playlists + get playlist by id
#featured_playlists, playlist

For later use:

current_user_saved_tracks
- Gets a list of the tracks saved in the current authorized user’s “Your Music” library

artist_top_tracks
- Get Spotify catalog information about an artist’s top 10 tracks by country. 

artist_related_artists
- Get Spotify catalog information about artists similar to an identified artist. Similarity is based on analysis of the Spotify community’s listening history.

playlist_add_items

recommendations(seed_artists=None, seed_genres=None, seed_tracks=None, limit=20, country=None, \*\*kwargs)
- Get a list of recommended tracks for one to five seeds. (at least one of seed_artists, seed_tracks and seed_genres are needed)

- Parameters:
- seed_artists - a list of artist IDs, URIs or URLs
- seed_tracks - a list of track IDs, URIs or URLs
- seed_genres - a list of genre names. Available genres for
- recommendations can be found by calling recommendation_genre_seeds

user_playlist_replace_tracks(user, playlist_id, tracks)
